In [6]:
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

In [7]:
train = pd.read_csv("/code/data/processed/proceeded_train_20200103_182856.csv", index_col=0)

In [8]:
train.head()

,Clip,Activity,Assessment,Game,session_title,count_correct_attempts,count_uncorrect_attempts,count_accuracy,acc_Cart Balancer (Assessment),acc_Cauldron Filler (Assessment),...,Magma Peak - Level 1,Treasure Map,Crystal Caves - Level 2,All Star Sorting,Tree Top City - Level 2,"Heavy, Heavier, Heaviest",Crystals Rule,Dino Drink,Air Show,Magma Peak - Level 2
0,11,3,0,4,32,0,0,0.0,-1.0,-1.0,...,1,0,0,153,1,0,0,0,0,0
1,14,4,1,6,31,1,0,1.0,-1.0,-1.0,...,1,1,0,153,1,0,61,0,180,0
2,14,4,2,6,32,1,11,0.5,-1.0,-1.0,...,1,1,0,153,1,0,61,0,180,0
3,24,9,4,10,32,2,11,0.5,-1.0,-1.0,...,2,1,0,203,2,0,61,91,180,1
4,28,10,5,13,31,3,12,0.5,-1.0,-1.0,...,2,3,0,203,2,0,212,91,295,1


In [9]:
group = train.installation_id
y = train.accuracy_group
X = train.drop('accuracy_group', axis=1)


In [18]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    np.random.seed(seed)
    # ラベルの数をカウント
    labels_num = np.max(y) + 1
    # 各グループのラベルの数をカウントする
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1
    # 各フォールドのラベルの数をカウント
    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)

    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)

    for i in range(k):
        test_k = i
        val_k = i+1 if i+1 != k else 0
        # print(val_k)
        train_groups = all_groups - groups_per_fold[test_k] #  - groups_per_fold[val_k]
        # val_groups = groups_per_fold[val_k]
        test_groups = groups_per_fold[test_k]
        # print(test_groups)
        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        # val_indices = [i for i, g in enumerate(groups) if g in val_groups]
        #test_indices = {str(g): [i for i, g in enumerate(groups) if g in test_groups]}
        
        test_indices = []
        n_g = None
        test_list = []
        for i, g in enumerate(groups):
            if g in test_groups:
                if n_g is not None and n_g != g:
                    test_indices.append(test_list)
                    test_list = []
                test_list.append(i)
                n_g = g

        test_indices = [np.random.choice(i, ) for i in test_indices]
        yield train_indices, test_indices, # val_indices,

In [19]:
a = stratified_group_k_fold(X=X, y=y, groups=group, k=5, seed=55)

In [20]:
for i, (train_ind, val_ind) in enumerate(a):
    print('------------------data------------------')
    a = val_ind
    # print(data)
    if i == 1:
        break

------------------data------------------
------------------data------------------


In [21]:
val_ind

[37,
 86,
 87,
 92,
 126,
 130,
 148,
 183,
 226,
 250,
 252,
 277,
 296,
 306,
 349,
 388,
 395,
 483,
 484,
 490,
 569,
 605,
 635,
 830,
 832,
 837,
 889,
 897,
 905,
 953,
 978,
 1074,
 1149,
 1182,
 1211,
 1220,
 1229,
 1232,
 1237,
 1257,
 1263,
 1267,
 1284,
 1293,
 1371,
 1372,
 1474,
 1515,
 1520,
 1555,
 1558,
 1581,
 1586,
 1594,
 1626,
 1670,
 1690,
 1720,
 1783,
 1809,
 1866,
 1886,
 1894,
 1906,
 1909,
 1912,
 1941,
 1956,
 1958,
 1984,
 1992,
 1999,
 2015,
 2098,
 2102,
 2125,
 2134,
 2203,
 2317,
 2348,
 2363,
 2470,
 2475,
 2479,
 2482,
 2556,
 2577,
 2615,
 2642,
 2656,
 2712,
 2713,
 2753,
 2760,
 2761,
 2792,
 2809,
 2915,
 2927,
 2930,
 2931,
 3005,
 3021,
 3036,
 3038,
 3039,
 3047,
 3103,
 3134,
 3161,
 3168,
 3186,
 3218,
 3294,
 3298,
 3303,
 3378,
 3532,
 3554,
 3562,
 3564,
 3568,
 3576,
 3578,
 3587,
 3603,
 3618,
 3621,
 3642,
 3672,
 3696,
 3726,
 3735,
 3760,
 3763,
 3780,
 3781,
 3819,
 3833,
 3836,
 3874,
 3885,
 3889,
 3891,
 3905,
 3929,
 4005,
 4012,

In [0]:
np.random.choice(val_ind['78cacfd1'])